<a href="https://colab.research.google.com/github/carloshvmoraes/CEB-IoT-5G/blob/main/ISGD_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating Smart Grid Devices into the Traditional Protection of Distribution Networks

## Install Python Libraries

In [2]:
!pip install pandapower

     |████████████████████████████████| 5.9 MB 7.2 MB/s 
  Created wheel for pandapower: filename=pandapower-2.8.0-py3-none-any.whl size=5881020 sha256=8a5f9e1027f512f3f13b4f44bcf589714eec1ab738c80affccc4fd3c43466706
  Stored in directory: /root/.cache/pip/wheels/8a/ce/91/cb5881926915c6c131434bde7bb43a9f9e1c7a54951ebd3dee
Successfully built pandapower


## Download Source ISGDTPDN

In [3]:
!wget https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/src/ISGDTPDN.py -O ISGDTPDN.py
!wget https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/examples/CWB_p64.py -O CWB_p64.py

--2022-03-03 19:26:07--  https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/src/ISGDTPDN.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2509 (2.5K) [text/plain]
Saving to: ‘ISGDTPDN.py’

ISGDTPDN.py         100%[===================>]   2.45K  --.-KB/s    in 0s      

2022-03-03 19:26:07 (40.0 MB/s) - ‘ISGDTPDN.py’ saved [2509/2509]

--2022-03-03 19:26:07--  https://raw.githubusercontent.com/carloshvmoraes/ISGDTPDN/main/examples/CWB_p64.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5433 (5.3K) [text/plain]
Savi

## Example - CWB Bus 64

<img src="https://github.com/carloshvmoraes/ISGDTPDN/raw/main/examples/CWB_p64.png" width="500" />

## Basic Use

In [8]:
from ISGDTPDN import CurrSaved, CurrSavedNew, FaultDetect
from CWB_p64 import new_network
import pandapower as pp
import random
import pandas as pd

random.seed(42)

# basic settings for 2 scenarios
scenarios = [
            {'sw_meters':[65], 'sw_remove':[64, 82, 83, 84], 'LoadRange':0.05},
            {'sw_meters':[64, 68], 'sw_remove':[], 'LoadRange':0.05},
            ]

for setting in scenarios:
  name_scenario = '-'.join(map(str,setting['sw_meters']))
  display(name_scenario)

  filename = name_scenario + '_' + str(setting['LoadRange']) + '_single'
  
  # create distribuition net
  net = new_network(medidores=setting['sw_meters'])
  
  # remove switchs before by switch meters
  if len(setting['sw_remove']) > 0:
    net.switch.drop(setting['sw_remove'])
  
  # load current before changes
  pp.runpp(net)
  I_Load = net.res_bus['p_mw'] * 72.46376811594203 / net.res_bus['vm_pu']
  
  # Fault Pattern Currents
  I_saved = CurrSaved(net)

  # load change by Range
  net.load['p_mw'] = [random.uniform(1-setting['LoadRange'], 1+setting['LoadRange']) * r['p_mw'] for _, r in net.load.iterrows()]

  # load current after changes
  pp.runpp(net)
  I_Load_new = net.res_bus['p_mw'] * 72.46376811594203 / net.res_bus['vm_pu']
  currLoad = pd.concat([I_Load, I_Load_new], axis=1, keys=['I Load', 'I Load New']).sort_index()
  currLoad.index.name = 'Bus'
  currLoad.to_csv(filename + '_iLoad.csv')

  # Fault Pattern correction
  I_saved_new = CurrSavedNew(net, I_saved)

  solutions = []
  for sw_id, r in net.switch.iterrows():
    # create fault
    net.switch.at[sw_id, 'closed'] = False
    pp.runpp(net, neglect_open_switch_branches=True)
    # remove fault
    net.switch.at[sw_id, 'closed'] = True
    
    # search fault
    fault_best = FaultDetect(net, I_saved_new)
    # best solution
    if sw_id in fault_best:
      index = fault_best.index(sw_id)
      res = {'fault':sw_id, 'choices': fault_best, 'position':(index+1)}
      solutions.append(res)

  dfsol = pd.DataFrame(data=solutions)
  dfsol = dfsol.set_index('fault').sort_index()
  dfsol.to_csv(filename + '_solutions.csv')
  display('Solutions:')
  display(dfsol)

'65'

'Solutions:'

,choices,position
fault,,
64,"[64, 65, 66, 67, 68, 69, 73, 78, 70, 74, 80, 8...",1
65,"[64, 65, 66, 67, 68, 69, 73, 78, 70, 74, 80, 8...",2
66,"[66, 67, 68, 64, 65, 69, 73, 78, 70, 74, 80, 8...",1
67,"[67, 68, 66, 64, 65, 69, 73, 78, 70, 74, 80, 8...",1
68,"[68, 67, 66, 69, 64, 65, 73, 78, 70, 74, 80, 8...",1
69,"[69, 73, 78, 70, 74, 80, 68, 81, 79, 75, 72, 6...",1
70,"[74, 70, 78, 80, 73, 81, 79, 75, 72, 77, 71, 7...",2
71,"[71, 76, 77, 82, 84, 83, 72, 75, 79, 81, 80, 7...",1
72,"[72, 75, 79, 81, 77, 71, 76, 80, 82, 84, 83, 7...",1


'64-68'

'Solutions:'

,choices,position
fault,,
64,"[64, 69, 65, 73, 74, 70, 66, 75, 67, 78, 76, 7...",1
65,"[65, 69, 66, 67, 68, 64, 73, 74, 70, 75, 78, 7...",1
66,"[66, 67, 68, 69, 65, 73, 74, 82, 70, 64, 75, 7...",1
67,"[67, 66, 68, 69, 65, 73, 74, 82, 70, 75, 64, 7...",1
68,"[68, 67, 66, 69, 65, 73, 82, 74, 70, 75, 78, 7...",1
69,"[69, 73, 82, 68, 74, 67, 66, 70, 75, 78, 79, 8...",1
70,"[70, 75, 78, 79, 80, 76, 77, 83, 84, 72, 71, 8...",1
71,"[81, 71, 72, 84, 77, 83, 76, 80, 79, 78, 75, 7...",2
72,"[72, 71, 81, 84, 77, 83, 76, 80, 79, 78, 75, 7...",1
